<a href="https://colab.research.google.com/github/Mathurshab2210/Ml_with_Mathur/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re# to search the words in doc
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
news=pd.read_csv('/content/drive/MyDrive/CNN/train.csv')

In [6]:
news.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
news.shape

(20800, 5)

In [9]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
 #missing value analysis
 news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
#  replacing null values
news=news.fillna('')

In [12]:
 #mergin author and text
 news['content']=news['author']+' '+news['title']
 print(news['content'])


0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
#seprating data and label
x=news.drop('label',axis=1)
y=news['label']

In [14]:
x,y

(          id                                              title  \
 0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2          2                  Why the Truth Might Get You Fired   
 3          3  15 Civilians Killed In Single US Airstrike Hav...   
 4          4  Iranian woman jailed for fictional unpublished...   
 ...      ...                                                ...   
 20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
 20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
 20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
 20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
 20799  20799                          What Keeps the F-35 Alive   
 
                                           author  \
 0                                  Darrell Lucus   
 1                                Daniel J. Flynn   
 2                     

Stemming:
means to find root word


In [15]:
port_stem=PorterStemmer()

In [16]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content)#to take alphabets and eexcluding everything else
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[port_stem.stem(word) for word in stem_content if not word in  stopwords.words('english')]
  stem_content=' '.join(stem_content)
  return stem_content

In [17]:
news['content']=news['content'].apply(stemming)

In [18]:
news['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [19]:
#seprating data and label
x=news['content'].values
y=news['label'].values

In [20]:
print(x,y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] [1 0 1 ... 0 1 1]


In [21]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [22]:
y.shape

(20800,)

In [23]:
#text to feature vector
vectorizer=TfidfVectorizer ()
vectorizer.fit(x)
X=vectorizer.transform(x)



In [24]:
X

<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

In [56]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, stratify=y, random_state=3)

In [57]:
 model=LogisticRegression()

In [58]:
model.fit(X_train,y_train)

LogisticRegression()

In [59]:
X_train_pred=model.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)

In [60]:
print(train_data_acc)

0.9861778846153846


In [61]:
X_test_pred=model.predict(X_test)
test_data_acc=accuracy_score(X_test_pred,y_test)

In [62]:
print('test acc:',test_data_acc)

test acc: 0.9764423076923077


In [63]:
#making a predictive sys

In [64]:
X_new=X_test[0]
pred=model.predict(X_new)

In [65]:
pred

array([1])

In [66]:
if pred[0] == 0:
  print("real news")
else:
  print('fake news')

fake news


In [67]:
y_test[0]

1

In [68]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
clf1=GaussianNB()
clf2= MultinomialNB()
clf3= BernoulliNB()
clf1.fit(X_train.toarray(),y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)



BernoulliNB()

In [69]:
X_train_pred=clf1.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for GaussianNB ')
X_test_pred=clf1.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' GaussianNB test acc:',test_data_acc)

0.9164663461538461 is training data accuracy for GaussianNB 
 GaussianNB test acc: 0.7949519230769231


In [70]:
X_train_pred=clf2.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for  MultinomialNB()')
X_test_pred=clf2.predict(X_test) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print('  MultinomialNB() test acc:',test_data_acc)

0.9763822115384615 is training data accuracy for  MultinomialNB()
  MultinomialNB() test acc: 0.9584134615384615


In [71]:
X_train_pred=clf3.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for BernoulliNB ')
X_test_pred=clf3.predict(X_test) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' BernoulliNB test acc:',test_data_acc)

0.9909254807692308 is training data accuracy for BernoulliNB 
 BernoulliNB test acc: 0.9788461538461538


In [72]:
X_new=X_test[0]
pred=clf3.predict(X_new)

In [73]:
pred

array([1])

In [74]:
y_test[0]

1